# Hermitage I5 migration notebook

This notebook app will run the ETL process to migrate Heremitage's I5 to a new Odoo 15 system.
It will read csv/tsv files from the input_csv_files, make the necessary transformations to make it importable in Odoo 15 and then load the data it into Hermitage's new Odoo 15 instance.

### Imports
- pandas: to make transformations on the data
- Models' migration config
- Import function

In [ ]:
import re
import pandas

from models_migration_config import models_migration_config

from import_functions import import_data, import_ignored_fields

INPUT_CSV_FILES_PATH = 'input_csv_files/'
GENERATED_CSV_FILES_PATH = 'generated_csv_files/'



**Transform Vendors from I5**

In [ ]:
vendors_file_path = f'{INPUT_CSV_FILES_PATH}APMASTER - Vendors.tsv'
vendors_dataframe = pandas.read_csv(vendors_file_path, sep='\t')
vendors_dataframe = vendors_dataframe.fillna('')
# AMVEND	AMNAME	AMADR1	AMADR2	AMADR3	AMTELE	AMFRGT	actual_term
# 1	BROAN MFG OR EDN	4641 PAYSPHERE CIRCLE	CHICAGO,IL 60674         		8778470145	1250	2% 10TH PROX
# 172	HUNTER FAN COMPANY	P O BOX 19773		PALATINE IL 60055-9773	9017441200	1000	1% 20TH
# 10003	LEDVANCE OR IMARK	P O BOX 72524	(OSRAM)	CLEVELAND OH 44192	8002555042	1000	2% 90Days
# 10004	ADVANCE TRANSFORMER CO	P O BOX 100332	ATLANTA GA  30384		0	750	2% 10TH PROX
# 10005	LUTRON ELECTRONICS CO INC	P O BOX 644396		PITTSBURGH, PA 15264-4396	8005239466	500	1%10TH PROX

col_names = ['AMVEND', 'AMNAME', 'AMADR1', 'AMADR2', 'AMADR3', 'AMTELE', 'AMFRGT', 'actual_term']
for col_name in col_names:
    vendors_dataframe[col_name] = vendors_dataframe[col_name].astype(str)
    vendors_dataframe[col_name] = vendors_dataframe[col_name].str.strip()

# Move AMADR1 column to the end for better visibility
vendors_dataframe = vendors_dataframe[['AMVEND', 'AMNAME', 'AMADR2', 'AMADR3', 'AMTELE', 'AMFRGT', 'actual_term', 'AMADR1']]
vendors_dataframe.rename(columns={'AMADR1': 'street'}, inplace=True)

vendors_dataframe['street2'] = ''
vendors_dataframe['city'] = ''
vendors_dataframe['state_code'] = ''
vendors_dataframe['zip'] = ''

def is_zip(zip_code : str) -> bool:
    """
    >>> is_zip('6067')
    True
    >>> is_zip('60674')
    True
    >>> is_zip('60674-1234')
    True
    """
    return bool(re.match(r'^\d{5}(-\d{4})?$', zip_code)) or bool(re.match(r'^\d{4}', zip_code))

us_states = {
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY',
    'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
    'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
}

def is_a_state(state : str) -> bool:
    return state in us_states

def clean_address_field(address_field: str) -> str:
    address_field = address_field.strip()
    address_field = re.sub(r'\s+', ' ', address_field)
    address_field = address_field.replace(',', ' ')
    return address_field


for row_index, row in vendors_dataframe.iterrows():
    row_address2 = row.AMADR2 or ''
    row_address2 = clean_address_field(row_address2)
    row_address3 = row.AMADR3 or ''
    row_address3 = clean_address_field(row_address3)

    if row_address3:
        row.street2 = row_address2
        city_state_zip = row_address3
    else:
        city_state_zip = row_address2
    if city_state_zip:
        city_state_zip_splitted = city_state_zip.split(' ')[::-1]

        for i, address_field in enumerate(city_state_zip_splitted):
            #import pdb;pdb.set_trace()
            if is_zip(address_field):
                row.zip = address_field.strip()
            elif is_a_state(address_field):
                row.state_code = address_field.strip()
            else:
                row.city = ' '.join(city_state_zip_splitted[i:][::-1]).strip()
                break
        if not row.zip or not row.state_code or not row.city and row.street2:
            city_state_zip_splitted = row.street2.split(' ')[::-1]
            for i, address_field in enumerate(city_state_zip_splitted):
                if not row.zip and is_zip(address_field):
                    row.zip = address_field.strip()
                elif not row.state_code and is_a_state(address_field):
                    row.state_code = address_field.strip()
                elif not row.city:
                    row.city = ' '.join(city_state_zip_splitted[i:][::-1]).strip()
                    break

states_dataframe = pandas.read_csv(f'{INPUT_CSV_FILES_PATH}state_codes.csv')
# Merge with states to get the state and country external id
vendors_dataframe = pandas.merge(vendors_dataframe, states_dataframe, on='state_code', how='left')

# Merge with the payment terms to get the payment term external id
payment_terms_dataframe = pandas.read_csv(f'{INPUT_CSV_FILES_PATH}account.payment.term.csv')
payment_terms_dataframe.rename(columns={'name': 'actual_term', 'id': 'property_supplier_payment_term_id/id'}, inplace=True)
merge_payment_terms_dataframe = payment_terms_dataframe[['actual_term', 'property_supplier_payment_term_id/id']]

vendors_dataframe = pandas.merge(vendors_dataframe, merge_payment_terms_dataframe, on='actual_term', how='left')
# remove unneeded columns
vendors_dataframe.drop(columns=['AMADR2', 'AMADR3', 'AMFRGT', 'actual_term', 'state_code'], inplace=True)
# rename columns to match the odoo model
vendors_dataframe.rename(columns={
    'AMVEND': 'vendor_account_number',
    'AMNAME': 'name',
    'AMTELE': 'phone'
}, inplace=True)

vendors_dataframe.insert(0, 'supplier_rank', 1)
# add Vendor tag
vendors_dataframe.insert(0, 'category_id/id', '__export__.res_partner_category_56_7fafda33')
# add id column populated with ids generated from 'i5_migration.res.partner.vendors_{row_index}'
vendors_dataframe.insert(0, 'id', [f'i5_migration.res.partner.vendor_{row_index}' for row_index in range(1, 1 + len(vendors_dataframe))])

vendors_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.vendors.csv', index=False, sep=',')


**Load vendors (res.partner)**

In [ ]:
import_data(file_csv='i5.vendors.csv', model_name='res.partner')